In [4]:
# use the MNE CLEAN data :')
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# some hyperparameter type things :)
sr = 200
t = 200
ch = 19
affix = '_mne_clean'

In [5]:
# load from .pkl files check affix !
raw_features = {}
raw_labels = {}
fileList = []

for letter in ['A', 'B', 'C', 'E', 'F', 'G', 'H', 'I']: 
    raw_features[f'subj_{letter}'] = np.load(f'pickles/subj_{letter}_features{affix}.npy')
    raw_labels[f'subj_{letter}'] = np.load(f'pickles/subj_{letter}_labels{affix}.npy').reshape(-1)
    fileList.append((f'subj_{letter}', raw_features[f'subj_{letter}'], raw_labels[f'subj_{letter}']))

In [3]:
# ConvNet_Finger

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

class ConvNet_Finger(nn.Module):
    def __init__(self):
        super(ConvNet_Finger, self).__init__()
        ### FILL IN ### [10 POINTS]

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, (7, 3)),
            nn.ReLU(), 
            #nn.MaxPool2d(2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 16, (5, 3)),
            nn.Conv2d(16, 32, (3, 3)),
            nn.ReLU(),
            #nn.MaxPool2d(2)
        )

        self.hidden = nn.Sequential(
            nn.Flatten(),
            # nn.Linear(121728, 256),
            nn.Linear(43456, 256),
            nn.ReLU(),
            nn.Dropout(0.4), # all features norefer
            # nn.Linear(1408, 256), #pseudosampled featrues .
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 2),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        ### FILL IN ### [5 POINTS]
        x = self.conv1(x)
        x = self.conv2(x)
        # x = self.conv3(x)
        x = self.hidden(x)
        return x

In [4]:
# final NeuralNet
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        ### FILL IN ### [10 POINTS]

        self.hidden = nn.Sequential(
            nn.Flatten(),
            # nn.Linear(121728, 256),
            nn.Linear(43456, 256),
            nn.ReLU(),
            nn.Dropout(0.4), # all features norefer
            # nn.Linear(1408, 256), #pseudosampled featrues .
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 5),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        nn.Sequential(
            nn.Linear(5, 16),
            nn.Linear(16, 32),
            nn.Linear(32, 5)
        )
        return x

In [3]:
# help :P

def predict(y_train):
    data = y_train
    pred = []
    with torch.no_grad():
        model.eval()
        data = data.to(device)
        predictions = model(data)
        _, predictions = predictions.max(1)
    return predictions

def evaluate(predictions, targets):
    n_samples, n_correct = 0, 0
    # print(targets.shape, predictions.shape)
    # print(type(targets), targets.shape, type(predictions), len(predictions))
    n_samples += targets.size(0)
    n_correct += (predictions == targets).sum().item()
    acc = n_correct / n_samples
    print(f'accuracy = {acc}')
    return acc

def binary_confusion(y_pred, target):
    # matrix represents [pred 0 targ 0, pred 0 targ 1] [pred 1 targ 0, pred 1 targ 1]
    matrix = [[0, 0],[0, 0]]
    for i in range(y_pred.shape[0]):
        #zero stands for not thumb
        if y_pred[i] == 0 and target[i] == 0:
            matrix[0][0] += 1
        elif y_pred[i] == 0 and target[i] == 1:
            matrix[0][1] += 1
        elif y_pred[i] == 1 and target[i] == 0:
            matrix[1][0] += 1
        elif y_pred[i] == 1 and target[i] == 1:
            matrix[1][1] += 1
    matrix = matrix/np.sum(matrix)
    return matrix

In [1]:
batch_size = 32
dropout = 0.2
n_epochs = 30
learning_rate = 1e-3
weight_decay = 1e-6
input_shape = (-1, 1, ch, t)

torch.cuda.set_device(4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sm = SMOTE()

# train neural net
accuracies = []
for subj, features, labels in fileList:
    new_features = []
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    for finger in range(5):
        y_train_ = np.where(y_train == finger, 1, 0)
        y_test_ = np.where(y_test == finger, 1, 0)
        X_res = X_train.reshape((X_train.shape[0], -1))
        X_res, y_res = sm.fit_resample(X_res, y_train_)
        X_res = X_res.reshape((X_res.shape[0], ch, -1))

        model = ConvNet_Finger().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        # Train the neural network

        # model.train()
        # data = X_res
        # targets = y_res
        # data = torch.tensor(data.reshape(int(data.shape[0]), 1, data.shape[1], data.shape[2]), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
        # data = data.to(device)
        # targets = torch.tensor(targets, dtype=torch.int64)
        # targets = targets.to(device)

        # for epoch in range(n_epochs):
        #     # print("epoch = ",epoch)
        #     model.train()

        #     # Forward pass
        #     predictions = model(data)
        #     loss = criterion(predictions, targets)

        #     # Backward pass
        #     optimizer.zero_grad()
        #     loss.backward()
        #     optimizer.step()
            
        # torch.save(model.state_dict(), f'pickles/models/{subj}_finger_{finger}.pt')
        
        model.load_state_dict(torch.load(f'pickles/models/{subj}_finger_{finger}.pt'))
        
        pred = predict(torch.as_tensor(X_train.reshape(input_shape), dtype=torch.float32).to(device))
        new_features.append(pred*finger)
    new_features = torch.stack(new_features)
    features = torch.transpose(torch.tensor(new_features, dtype=torch.float32), 0, 1).to(device).requires_grad_(True)
    print(features.shape, features)
    
    # final NeuralNet
    model = NeuralNet().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Train the neural network
    model.train()
    data = features
    targets = torch.tensor(y_train, dtype=torch.int64).to(device)

    for epoch in range(n_epochs):
        # print("epoch = ",epoch)
        model.train()

        # Forward pass
        predictions = model(data)
        loss = criterion(predictions, targets)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    torch.save(model.state_dict(), f'pickles/models/{subj}_final.pt')

    print(f'trained {subj} {finger}')
    new_features = []

    for finger in range(5):
        model = ConvNet_Finger().to(device)
        criterion = nn.CrossEntropyLoss
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        model.load_state_dict(torch.load(f'pickles/models/{subj}_finger_{finger}.pt'))

        y_res = np.where(y_test == finger, 1, 0)
        y_res = torch.tensor(y_res, dtype=torch.int64).to(device)
        pred = predict(torch.tensor(X_test.reshape(input_shape), dtype=torch.float32).to(device))
        print(f'accuracy {subj} finger {finger} = {evaluate(pred, y_res)} {binary_confusion(pred, y_res)})')
        new_features.append(pred*finger)


    new_features = torch.stack(new_features)
    features = torch.transpose(torch.tensor(new_features, dtype=torch.float32), 0, 1).to(device).requires_grad_(True)
    
    model = NeuralNet().to(device)
    model.load_state_dict(torch.load(f'pickles/models/{subj}_final.pt'))
    acc = evaluate(predict(features), torch.tensor(y_test, dtype=torch.int64).to(device))
    print(f'FINAL accuracy {subj} = {acc} ***************************************')
    accuracies.append(acc)

print(f'avg accuracy = {np.mean(accuracies)}')

NameError: name 'ch' is not defined

In [4]:
# svm on fingers
from sklearn import svm
from sklearn import metrics
from sklearn.ensemble import BaggingClassifier
import pickle

sm = SMOTE()

accuracies = []
for subj, features, labels in fileList:
    new_features = []
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    
    for finger in range(5):
        y_train_ = np.where(y_train == finger, 1, 0)
        y_test_ = np.where(y_test == finger, 1, 0)
        X_res = X_train.reshape((X_train.shape[0], -1))
        X_res, y_res = sm.fit_resample(X_res, y_train_)

        clf = BaggingClassifier(svm.SVC(kernel='rbf'), n_estimators=20)
        clf.fit(X_res, y_res)
        # clf = pickle.load(open(f'pickles/models/{subj}_finger_{finger}_svm.pkl', 'rb'))
        pickle.dump(clf, open(f'pickles/models/{subj}_finger_{finger}_svm.pkl', 'wb'))
        pred = clf.predict(X_train.reshape((X_train.shape[0], -1)))
        new_features.append(pred)
    features = np.array(new_features).transpose()
    print(features.shape)
    
    classifier = BaggingClassifier(svm.SVC(kernel='rbf'), n_estimators=20)
    classifier.fit(features, y_train)

    print(f'trained {subj} {finger}')
    new_features = []

    # test
    for finger in range(5):
        clf = pickle.load(open(f'pickles/models/{subj}_finger_{finger}_svm.pkl', 'rb'))
        y_res = np.where(y_test == finger, 1, 0)
        pred = clf.predict(X_test.reshape((X_test.shape[0], -1)))
        print(f'accuracy {subj} finger {finger} = {metrics.accuracy_score(y_res, pred)} {binary_confusion(pred, y_res)})')
        new_features.append(pred*finger)

    features = np.array(new_features).transpose()

    pred = classifier.predict(features)
    acc = metrics.accuracy_score(y_test, pred)
    print(f'FINAL accuracy {subj} = {acc} ***************************************')
    accuracies.append(acc)
print(f'avg accuracy = {np.mean(accuracies)}')

(1278, 5)
trained subj_A 4
accuracy subj_A finger 0 = 0.6824817518248175 [[0.56751825 0.09489051]
 [0.22262774 0.1149635 ]])
accuracy subj_A finger 1 = 0.6934306569343066 [[0.6459854  0.10583942]
 [0.20072993 0.04744526]])
accuracy subj_A finger 2 = 0.6970802919708029 [[0.61131387 0.12591241]
 [0.1770073  0.08576642]])
accuracy subj_A finger 3 = 0.6678832116788321 [[0.58576642 0.11861314]
 [0.21350365 0.08211679]])
accuracy subj_A finger 4 = 0.7116788321167883 [[0.62956204 0.14233577]
 [0.1459854  0.08211679]])
FINAL accuracy subj_A = 0.3102189781021898 ***************************************
(2667, 5)
trained subj_B 4
accuracy subj_B finger 0 = 0.8286713286713286 [[0.72902098 0.11975524]
 [0.05157343 0.09965035]])
accuracy subj_B finger 1 = 0.798951048951049 [[0.74562937 0.13548951]
 [0.06555944 0.05332168]])
accuracy subj_B finger 2 = 0.763986013986014 [[0.71590909 0.13374126]
 [0.10227273 0.04807692]])
accuracy subj_B finger 3 = 0.7963286713286714 [[0.70979021 0.11975524]
 [0.083916

In [21]:
# ensemble w/ cnn + svm 
from sklearn import svm
from sklearn import metrics

batch_size = 32
dropout = 0.2
n_epochs = 30
learning_rate = 1e-3
weight_decay = 1e-6
input_shape = (-1, 1, ch, t)

torch.cuda.set_device(4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sm = SMOTE()

# train neural net
accuracies = []
for subj, features, labels in fileList:
    new_features = []
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    for finger in range(5):
        y_train_ = np.where(y_train == finger, 1, 0)
        y_test_ = np.where(y_test == finger, 1, 0)
        X_res = X_train.reshape((X_train.shape[0], -1))
        X_res, y_res = sm.fit_resample(X_res, y_train_)
        X_res = X_res.reshape((X_res.shape[0], ch, -1))

        model = ConvNet_Finger().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        # Train the neural network

        model.train()
        data = X_res
        targets = y_res
        data = torch.tensor(data.reshape(int(data.shape[0]), 1, data.shape[1], data.shape[2]), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
        data = data.to(device)
        targets = torch.tensor(targets, dtype=torch.int64)
        targets = targets.to(device)

        for epoch in range(n_epochs):
            # print("epoch = ",epoch)
            model.train()

            # Forward pass
            predictions = model(data)
            loss = criterion(predictions, targets)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        torch.save(model.state_dict(), f'pickles/models/{subj}_finger_{finger}.pt')
        
        pred = predict(torch.as_tensor(X_train.reshape(input_shape), dtype=torch.float32).to(device))
        new_features.append(pred.to('cpu')*finger)
    features = np.array(new_features).transpose()

    # final NeuralNet
    classifier = svm.SVC(kernel='rbf')
    classifier.fit(features, y_train)

    print(f'trained {subj} {finger}')
    new_features = []

    for finger in range(5):
        model = ConvNet_Finger().to(device)
        criterion = nn.CrossEntropyLoss
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        model.load_state_dict(torch.load(f'pickles/models/{subj}_finger_{finger}.pt'))

        y_res = np.where(y_test == finger, 1, 0)
        y_res = torch.tensor(y_res, dtype=torch.int64).to(device)
        pred = predict(torch.tensor(X_test.reshape(input_shape), dtype=torch.float32).to(device))
        print(f'accuracy {subj} finger {finger} = {evaluate(pred, y_res)} {binary_confusion(pred, y_res)})')
        new_features.append(pred.to('cpu')*finger)

    features = np.array(new_features).transpose()

    pred = classifier.predict(features)
    acc = metrics.accuracy_score(y_test, pred)
    print(f'FINAL accuracy {subj} = {acc} ***************************************')
    accuracies.append(acc)

print(f'avg accuracy = {np.mean(accuracies)}')

trained subj_A 4
accuracy = 0.7828467153284672
accuracy subj_A finger 0 = 0.7828467153284672 [[0.69890511 0.12043796]
 [0.09671533 0.08394161]])
accuracy = 0.791970802919708
accuracy subj_A finger 1 = 0.791970802919708 [[0.75       0.16240876]
 [0.04562044 0.0419708 ]])
accuracy = 0.7846715328467153
accuracy subj_A finger 2 = 0.7846715328467153 [[0.72262774 0.13138686]
 [0.08394161 0.0620438 ]])
accuracy = 0.7737226277372263
accuracy subj_A finger 3 = 0.7737226277372263 [[0.71350365 0.15145985]
 [0.07481752 0.06021898]])
accuracy = 0.7956204379562044
accuracy subj_A finger 4 = 0.7956204379562044 [[0.73722628 0.12773723]
 [0.07664234 0.05839416]])
FINAL accuracy subj_A = 0.3467153284671533 ***************************************
trained subj_B 4
accuracy = 0.791958041958042
accuracy subj_B finger 0 = 0.791958041958042 [[0.69318182 0.13636364]
 [0.07167832 0.09877622]])
accuracy = 0.8085664335664335
accuracy subj_B finger 1 = 0.8085664335664335 [[0.80856643 0.19143357]
 [0.         0.   

In [22]:
# helper code for convformer (2 class :D)

import argparse
import os
gpus = [0]
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(map(str, gpus))
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.autograd import Variable
# from torchsummary import summary
import torch.autograd as autograd
from torchvision.models import vgg19

import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.nn.init as init

from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
from sklearn.decomposition import PCA

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
# from common_spatial_pattern import csp

import matplotlib.pyplot as plt
# from torch.utils.tensorboard import SummaryWriter
from torch.backends import cudnn
cudnn.benchmark = False
cudnn.deterministic = True

# Convolution module
# use conv to capture local features, instead of postion embedding.
class PatchEmbedding(nn.Module):
    def __init__(self, emb_size=40):
        # self.patch_size = patch_size
        super().__init__()

        self.shallownet = nn.Sequential(
            nn.Conv2d(1, 40, (1, 20), (1, 1)),
            nn.Conv2d(40, 40, (19, 1), (1, 1)),
            nn.BatchNorm2d(40),
            nn.ELU(),
            nn.AvgPool2d((1, 75), (1, 15)),  # pooling acts as slicing to obtain 'patch' along the time dimension as in ViT
            nn.Dropout(0.5),
        )

        self.projection = nn.Sequential(
            nn.Conv2d(40, emb_size, (1, 1), stride=(1, 1)),  # transpose, conv could enhance fiting ability slightly
            Rearrange('b e (h) (w) -> b (h w) e'),
        )


    def forward(self, x: Tensor) -> Tensor:
        b, _, _, _ = x.shape
        x = self.shallownet(x)
        x = self.projection(x)
        return x


class MultiHeadAttention(nn.Module):
    def __init__(self, emb_size, num_heads, dropout):
        super().__init__()
        self.emb_size = emb_size
        self.num_heads = num_heads
        self.keys = nn.Linear(emb_size, emb_size)
        self.queries = nn.Linear(emb_size, emb_size)
        self.values = nn.Linear(emb_size, emb_size)
        self.att_drop = nn.Dropout(dropout)
        self.projection = nn.Linear(emb_size, emb_size)

    def forward(self, x: Tensor, mask: Tensor = None) -> Tensor:
        queries = rearrange(self.queries(x), "b n (h d) -> b h n d", h=self.num_heads)
        keys = rearrange(self.keys(x), "b n (h d) -> b h n d", h=self.num_heads)
        values = rearrange(self.values(x), "b n (h d) -> b h n d", h=self.num_heads)
        energy = torch.einsum('bhqd, bhkd -> bhqk', queries, keys)  
        if mask is not None:
            fill_value = torch.finfo(torch.float32).min
            energy.mask_fill(~mask, fill_value)

        scaling = self.emb_size ** (1 / 2)
        att = F.softmax(energy / scaling, dim=-1)
        att = self.att_drop(att)
        out = torch.einsum('bhal, bhlv -> bhav ', att, values)
        out = rearrange(out, "b h n d -> b n (h d)")
        out = self.projection(out)
        return out


class ResidualAdd(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        res = x
        x = self.fn(x, **kwargs)
        x += res
        return x


class FeedForwardBlock(nn.Sequential):
    def __init__(self, emb_size, expansion, drop_p):
        super().__init__(
            nn.Linear(emb_size, expansion * emb_size),
            nn.GELU(),
            nn.Dropout(drop_p),
            nn.Linear(expansion * emb_size, emb_size),
        )


class GELU(nn.Module):
    def forward(self, input: Tensor) -> Tensor:
        return input*0.5*(1.0+torch.erf(input/math.sqrt(2.0)))


class TransformerEncoderBlock(nn.Sequential):
    def __init__(self,
                 emb_size,
                 num_heads=10,
                 drop_p=0.5,
                 forward_expansion=4,
                 forward_drop_p=0.5):
        super().__init__(
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                MultiHeadAttention(emb_size, num_heads, drop_p),
                nn.Dropout(drop_p)
            )),
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                FeedForwardBlock(
                    emb_size, expansion=forward_expansion, drop_p=forward_drop_p),
                nn.Dropout(drop_p)
            )
            ))


class TransformerEncoder(nn.Sequential):
    def __init__(self, depth, emb_size):
        super().__init__(*[TransformerEncoderBlock(emb_size) for _ in range(depth)])


class ClassificationHead(nn.Sequential):
    def __init__(self, emb_size, n_classes):
        super().__init__()
        
        # global average pooling
        self.clshead = nn.Sequential(
            Reduce('b n e -> b e', reduction='mean'),
            nn.LayerNorm(emb_size),
            nn.Linear(emb_size, n_classes)
        )
        self.fc = nn.Sequential(
            nn.Linear(320, 256),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(256, 32),
            nn.ELU(),
            nn.Dropout(0.3),
            nn.Linear(32, 2) # i changed this to 2 because 2 classes .. ?
        )

    def forward(self, x):
        x = x.contiguous().view(x.size(0), -1)
        out = self.fc(x)
        return x, out


class Conformer(nn.Sequential):
    def __init__(self, emb_size=40, depth=6, n_classes=4, **kwargs):
        super().__init__(

            PatchEmbedding(emb_size),
            TransformerEncoder(depth, emb_size),
            ClassificationHead(emb_size, n_classes)
        )

def get_data(features, labels):      
    features = features.reshape(features.shape[0], 1, features.shape[1], features.shape[2])

    # split and shuffle data
    train_data, test_data, train_label, test_label = train_test_split(features, labels, test_size=0.3)

    shuffle_num = np.random.permutation(len(train_data))
    train_data = train_data[shuffle_num, :, :, :]
    train_label = train_label[shuffle_num]

    # standardize #UMMM SHOULD I BE DOING THIS ?????
    target_mean = np.mean(train_data)
    target_std = np.std(train_data)
    train_data = (train_data- target_mean) / target_std
    test_data = (test_data - target_mean) / target_std

    # data shape: (trial, conv channel, electrode channel, time samples)
    return train_data, test_data, train_label, test_label

class ExP():
    def __init__(self, subj):
        super(ExP, self).__init__()
        self.batch_size = 64
        self.n_epochs = 200
        self.c_dim = 5 # ummm does this refer to classes ? who fking knows lol
        self.lr = 0.0002
        self.b1 = 0.5 # wtf do these mean lol :P
        self.b2 = 0.999
        self.dimension = (19, 200) # erm. awesum used to be (190, 50) i changed ? but dunno if that makes sense tbh
        self.subj = subj # pass in the name of the subject (?) because each subject gets a diff model :D

        self.start_epoch = 0
        self.root = '/Preprocessing/'

        # self.log_write = open("./results/log_subject%d.txt" % self.subjList, "w") #ERMMMMMMM

        self.Tensor = torch.cuda.FloatTensor
        self.LongTensor = torch.cuda.LongTensor

        self.criterion_l1 = torch.nn.L1Loss().cuda()
        self.criterion_l2 = torch.nn.MSELoss().cuda()
        self.criterion_cls = torch.nn.CrossEntropyLoss().cuda()

        self.model = Conformer().cuda()
        self.model = nn.DataParallel(self.model, device_ids=[i for i in range(len(gpus))])
        self.model = self.model.cuda()
        # summary(self.model, (1, 22, 1000))


    # Segmentation and Reconstruction (S&R) data augmentation
    def interaug(self, timg, label):  
        aug_data = []
        aug_label = []
        for cls4aug in range(5):
            cls_idx = np.where(label == cls4aug)
            tmp_data = timg[cls_idx]
            tmp_label = label[cls_idx]

            tmp_aug_data = np.zeros((int(self.batch_size / 4), 1, 19, 200))
        
            for ri in range(int(self.batch_size / 4)):
                for rj in range(8):
                    print(ri, rj)
                    rand_idx = np.random.randint(0, tmp_data.shape[0], 8)
                    wtf = tmp_data[rand_idx[rj], :, :, rj * 125:(rj + 1) * 125]
                    print(wtf.shape)
                    tmp_aug_data[ri, :, :, rj * 125:(rj + 1) * 125] = tmp_data[rand_idx[rj], :, :,
                                                                      rj * 125:(rj + 1) * 125]

            aug_data.append(tmp_aug_data)
            aug_label.append(tmp_label[:int(self.batch_size / 4)])
        aug_data = np.concatenate(aug_data)
        aug_label = np.concatenate(aug_label)
        aug_shuffle = np.random.permutation(len(aug_data))
        aug_data = aug_data[aug_shuffle, :, :]
        aug_label = aug_label[aug_shuffle]

        aug_data = torch.from_numpy(aug_data).cuda()
        aug_data = aug_data.float()
        aug_label = torch.from_numpy(aug_label-1).cuda()
        aug_label = aug_label.long()
        return aug_data, aug_label

    def load_data(self, train_data, test_data, train_label, test_label): # reading the specific subject's data from mneList
        self.train_data = train_data
        self.test_data = test_data
        self.train_label = train_label
        self.test_label = test_label

    def train(self):
        img = torch.from_numpy(self.train_data) 
        label = torch.from_numpy(self.train_label) 

        dataset = torch.utils.data.TensorDataset(img, label)
        self.dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=self.batch_size, shuffle=True)

        test_data = torch.from_numpy(self.test_data)
        test_label = torch.from_numpy(self.test_label)
        test_dataset = torch.utils.data.TensorDataset(test_data, test_label)
        self.test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=self.batch_size, shuffle=True)

        # Optimizers
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, betas=(self.b1, self.b2))

        test_data = Variable(test_data.type(self.Tensor))
        test_label = Variable(test_label.type(self.LongTensor))

        bestAcc = 0
        averAcc = 0
        num = 0
        Y_true = 0
        Y_pred = 0

        # Train the cnn model
        total_step = len(self.dataloader)
        curr_lr = self.lr

        for e in range(self.n_epochs):
            # in_epoch = time.time()
            self.model.train()
            for i, (img, label) in enumerate(self.dataloader):

                img = Variable(img.cuda().type(self.Tensor))
                label = Variable(label.cuda().type(self.LongTensor))

                # data augmentation # skipping this for now cuz i tbh dunno whats going on 
                # aug_data, aug_label = self.interaug(self.train_data, self.train_label)
                # img = torch.cat((img, aug_data))
                # label = torch.cat((label, aug_label))


                tok, outputs = self.model(img)

                loss = self.criterion_cls(outputs, label) 

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()


            # out_epoch = time.time()


            # test process
            if (e + 1) % 1 == 0:
                self.model.eval()
                Tok, Cls = self.model(test_data)


                loss_test = self.criterion_cls(Cls, test_label)
                y_pred = torch.max(Cls, 1)[1]
                acc = float((y_pred == test_label).cpu().numpy().astype(int).sum()) / float(test_label.size(0))
                train_pred = torch.max(outputs, 1)[1]
                train_acc = float((train_pred == label).cpu().numpy().astype(int).sum()) / float(label.size(0))

                print('Epoch:', e,
                      '  Train loss: %.6f' % loss.detach().cpu().numpy(),
                      '  Test loss: %.6f' % loss_test.detach().cpu().numpy(),
                      '  Train accuracy %.6f' % train_acc,
                      '  Test accuracy is %.6f' % acc)

                # self.log_write.write(str(e) + "    " + str(acc) + "\n")
                num = num + 1
                averAcc = averAcc + acc
                if acc > bestAcc:
                    bestAcc = acc
                    Y_true = test_label
                    Y_pred = y_pred
                    
        return (bestAcc, averAcc / num, self.model, Y_pred)


In [23]:
torch.cuda.set_device(0)
faverage = []
sm = SMOTE()
for subj, features, labels in fileList:
    new_features = []
    faverage = []
    train_data, test_data, train_label, test_label = get_data(features, labels)
    for finger in range(5):
        y_train = np.where(train_label == finger, 1, 0)
        y_test = np.where(test_label == finger, 1, 0)
        X_res = train_data.reshape((train_data.shape[0], -1))
        X_res, y_res = sm.fit_resample(X_res, y_train)
        X_res = X_res.reshape((X_res.shape[0], 1, ch, -1))

        conformer = ExP(subj)
        conformer.load_data(X_res, test_data, y_res, y_test)
        bestAcc, averAcc, model = conformer.train()
        torch.save(model.state_dict(), f'pickles/models/{subj}_finger_{finger}_conformer.pt')        

        print(f'{subj}: Best Acc: {bestAcc} Average acc: {averAcc} *************************************')
        # faverage.append(averAcc)
    
# print(f'final average acc: {np.mean(faverage)}')
# okay load data
# model uses self.get_source_data() to pull data from matlab files.


Epoch: 0   Train loss: 0.624363   Test loss: 0.686735   Train accuracy 1.000000   Test accuracy is 0.518248
Epoch: 1   Train loss: 0.601396   Test loss: 0.656739   Train accuracy 1.000000   Test accuracy is 0.609489
Epoch: 2   Train loss: 0.538258   Test loss: 0.618612   Train accuracy 0.500000   Test accuracy is 0.671533
Epoch: 3   Train loss: 0.262282   Test loss: 0.788154   Train accuracy 1.000000   Test accuracy is 0.474453
Epoch: 4   Train loss: 0.337729   Test loss: 0.792381   Train accuracy 1.000000   Test accuracy is 0.547445
Epoch: 5   Train loss: 0.494500   Test loss: 0.618598   Train accuracy 1.000000   Test accuracy is 0.677007
Epoch: 6   Train loss: 0.451187   Test loss: 0.990456   Train accuracy 1.000000   Test accuracy is 0.556569
Epoch: 7   Train loss: 0.531090   Test loss: 0.763332   Train accuracy 1.000000   Test accuracy is 0.600365
Epoch: 8   Train loss: 0.743631   Test loss: 0.928613   Train accuracy 0.500000   Test accuracy is 0.556569
Epoch: 9   Train loss: 0.636

In [ ]:
torch.cuda.set_device(0)
faverage = []
sm = SMOTE()
for subj, features, labels in fileList:
    new_features = []
    faverage = []
    train_data, test_data, train_label, test_label = get_data(features, labels)
    for finger in range(5):
        y_train = np.where(train_label == finger, 1, 0)
        y_test = np.where(test_label == finger, 1, 0)
        X_res = train_data.reshape((train_data.shape[0], -1))
        X_res, y_res = sm.fit_resample(X_res, y_train)
        X_res = X_res.reshape((X_res.shape[0], 1, ch, -1))

        conformer = ExP(subj)
        conformer.load_data(X_res, test_data, y_res, y_test)
        # bestAcc, averAcc, model, y_pred = conformer.train()
        # torch.save(model.state_dict(), f'pickles/models/{subj}_finger_{finger}_conformer.pt')        

        y_pred = conformer.load_state_dict(torch.load(f'pickles/models/{subj}_finger_{finger}_conformer.pt'))
        new_features.append(pred.to('cpu') * finger)
        # print(f'{subj}: Best Acc: {bestAcc} Average acc: {averAcc} *************************************')
        # faverage.append(averAcc)
    
# print(f'final average acc: {np.mean(faverage)}')
# okay load data
# model uses self.get_source_data() to pull data from matlab files.
